In [1]:
import pandas as pd

In [2]:
election_round = "runoff"

df = pd.read_excel(f"../../data/Chile/2021_governor_{election_round}.xlsx", sheet_name="Votación", header=6)
df.head()

,Nro Región,Región,Circunscripción senatorial,Distrito,Comuna,Circunscripción electoral,Local,Mesa,Lista,Pacto,Partido,Nro.Voto,Nombres,Primer apellido,Segundo apellido,Votos,Cargo
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,11M,XS,FRENTE AMPLIO,COMUNES,200.0,JOSE MIGUEL,CARVAJAL,GALLARDO,23,ELECTO
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,11M,YN,UNIDAD CONSTITUYENTE,INDEPENDIENTES-LISTA YN,202.0,MARCO ANTONIO,PEREZ,BARRIA,18,NaN
2,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,11M,NaN,NaN,NaN,NaN,VOTOS NULOS,NaN,NaN,0,NaN
3,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,11M,NaN,NaN,NaN,NaN,VOTOS EN BLANCO,NaN,NaN,0,NaN
4,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,12M,XS,FRENTE AMPLIO,COMUNES,200.0,JOSE MIGUEL,CARVAJAL,GALLARDO,39,ELECTO


In [3]:
df["Candidato"] = df["Nombres"] + " " + df["Primer apellido"] + " " + df["Segundo apellido"]
def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

cols = ["Región", "Comuna", "Local", "Mesa"]
df["polling_id"] = create_polling_id(df, columns=cols)

In [4]:
data = df[["polling_id", "Candidato", "Partido", "Pacto", "Votos"]].dropna(subset=["Pacto"])
data = data.rename(columns={"Pacto": "coalition", "Candidato": "candidate", "Partido": "party", "Votos": "value"})
data["value"] = data["value"].astype(str).apply(lambda x: x.strip()).apply(lambda x: 0 if x == "" else x).astype(float)

tt = data.groupby(["polling_id", "coalition", "party", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt["flag_candidates"] = 1

tt.head()

,polling_id,coalition,party,candidate,value,rate,flag_candidates
0,DE ANTOFAGASTA-ANTOFAGASTA-CENTRO DE EDUCACION...,CHILE VAMOS,RENOVACION NACIONAL,MARCO ANTONIO DIAZ MUÑOZ,16.0,0.301887,1
1,DE ANTOFAGASTA-ANTOFAGASTA-CENTRO DE EDUCACION...,UNIDAD CONSTITUYENTE,INDEPENDIENTES-LISTA YN,RICARDO HERIBERTO DIAZ CORTES,37.0,0.698113,1
2,DE ANTOFAGASTA-ANTOFAGASTA-CENTRO DE EDUCACION...,CHILE VAMOS,RENOVACION NACIONAL,MARCO ANTONIO DIAZ MUÑOZ,17.0,0.298246,1
3,DE ANTOFAGASTA-ANTOFAGASTA-CENTRO DE EDUCACION...,UNIDAD CONSTITUYENTE,INDEPENDIENTES-LISTA YN,RICARDO HERIBERTO DIAZ CORTES,40.0,0.701754,1
4,DE ANTOFAGASTA-ANTOFAGASTA-CENTRO DE EDUCACION...,CHILE VAMOS,RENOVACION NACIONAL,MARCO ANTONIO DIAZ MUÑOZ,18.0,0.391304,1


In [5]:
df_location = df[["polling_id", "Región", "Comuna", "Local", "Mesa"]].drop_duplicates()
df_location = df_location.rename(columns={
    "Región": "region",
    "Comuna": "commune",
    "Local": "place",
    "Mesa": "polling_station"
})

In [6]:
election_type = "governor"
if election_round == "runoff":
  election_type = "governor2"

tt.to_csv(f"../../data_output/Chile/2021_{election_type}_polling_station.csv.gz")
df_location.to_csv(f"../../data_output/Chile/2021_{election_type}_polling_station_location.csv.gz")